In [1]:
import json
from pathlib import Path
import re

In [2]:
input_file = Path('WaveFlow/utils/open_random_segment.ipynb')

In [3]:
input_file.stem

'open_random_segment'

In [4]:
output_path = Path(input_file.stem)
output_path

PosixPath('open_random_segment')

In [11]:
input_nb = json.load(open(input_file,'r',encoding="utf-8"))

In [12]:
input_nb

{'cells': [{'cell_type': 'code',
   'execution_count': 15,
   'metadata': {},
   'outputs': [],
   'source': ['#export\n',
    'import json\n',
    'import re\n',
    'import fire\n',
    'from pathlib import Path\n',
    '\n',
    'def is_tests(cell):\n',
    "    return cell['source'] and re.match(r'^#tests$', cell['source'][0])\n",
    '\n',
    'def blankt_cells(cells):\n',
    '    output_cells = []\n',
    '    for cell in cells:\n',
    "        if cell['cell_type'] == 'code' and not is_tests(cell):\n",
    "            cell['source'] = [code_line if code_line.strip().startswith('#') \n",
    "                              else '\\n' if code_line.endswith('\\n') \n",
    "                              else '' \n",
    "                              for code_line in cell['source']]\n",
    '        output_cells.append(cell)\n',
    '    return output_cells\n',
    '\n',
    'def save_path(input_path, version):\n',
    "    name_unblnkt = input_path.stem.split('-blnkt')[0]\n",
   

In [14]:
def is_export(cell):
    if cell['cell_type'] != 'code': return False
    src = cell['source']
    if len(src) == 0 or len(src[0]) < 7: return False
    return re.match(r'^#export$', src[0], re.IGNORECASE) is not None

In [15]:
export_cells = [c for c in input_nb['cells'] if is_export(c)]

In [16]:
export_cells

[{'cell_type': 'code',
  'execution_count': 15,
  'metadata': {},
  'outputs': [],
  'source': ['#export\n',
   'import json\n',
   'import re\n',
   'import fire\n',
   'from pathlib import Path\n',
   '\n',
   'def is_tests(cell):\n',
   "    return cell['source'] and re.match(r'^#tests$', cell['source'][0])\n",
   '\n',
   'def blankt_cells(cells):\n',
   '    output_cells = []\n',
   '    for cell in cells:\n',
   "        if cell['cell_type'] == 'code' and not is_tests(cell):\n",
   "            cell['source'] = [code_line if code_line.strip().startswith('#') \n",
   "                              else '\\n' if code_line.endswith('\\n') \n",
   "                              else '' \n",
   "                              for code_line in cell['source']]\n",
   '        output_cells.append(cell)\n',
   '    return output_cells\n',
   '\n',
   'def save_path(input_path, version):\n',
   "    name_unblnkt = input_path.stem.split('-blnkt')[0]\n",
   "    return input_path.with_name(f'

In [23]:
out_content = '''#!/usr/bin/env python3
# coding: utf-8\n\n'''

for cell in export_cells:
    out_content += ''.join(cell['source'][1:]) + '\n\n'

In [25]:
out_content = re.sub(r' +$', '', out_content, flags=re.MULTILINE)

In [26]:
print(out_content)

#!/usr/bin/env python3
# coding: utf-8

import json
import re
import fire
from pathlib import Path

def is_tests(cell):
    return cell['source'] and re.match(r'^#tests$', cell['source'][0])

def blankt_cells(cells):
    output_cells = []
    for cell in cells:
        if cell['cell_type'] == 'code' and not is_tests(cell):
            cell['source'] = [code_line if code_line.strip().startswith('#')
                              else '\n' if code_line.endswith('\n')
                              else ''
                              for code_line in cell['source']]
        output_cells.append(cell)
    return output_cells

def save_path(input_path, version):
    name_unblnkt = input_path.stem.split('-blnkt')[0]
    return input_path.with_name(f'{name_unblnkt}-blnkt-{version:02}{input_path.suffix}')

def blankt_file(input_path, verbose):
    input_file = json.load(open(input_path,'r',encoding="utf-8"))

    input_file['cells'] = blankt_cells(input_file['cells'])
    if verbose:
        p

In [30]:
with io.open(output_path, "w", encoding="utf-8") as f:
    f.write(out_content[:-2])